# 2.7 学習と検証の実施

- 本ファイルでは、SSDの学習と検証の実施を行います。手元のマシンで動作を確認後、AWSのGPUマシンで計算します。
- p2.xlargeで約6時間かかります。


# 学習目標

1.	SSDの学習を実装できるようになる

# 事前準備

- AWS EC2 のGPUインスタンスを使用します
- フォルダ「utils」のssd_model.pyをします

In [1]:
# パッケージのimport
import cv2
import numpy as np
import os.path as osp
import time
import random
import pandas as pd

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim


In [2]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

使用デバイス： cuda:0


# DatasetとDataLoaderを作成する

In [4]:
from utils.ssd_model import make_datapath_list, VOCDataset, DataTransform, Anno_xml2list, od_collate_fn


# ファイルパスのリストを取得
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath)

# Datasetを作成
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)  # (BGR)の色の平均値
input_size = 300  # 画像のinputサイズを300×300にする

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))


# DataLoaderを作成する
batch_size = 32  # 32

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


# ネットワークモデルの作成する

In [5]:
from utils.ssd_model import SSD

# SSD300の設定
ssd_cfg = {
    'num_classes': 21,  # 背景クラスを含めた合計クラス数
    'input_size': 300,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 出力するDBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# SSDネットワークモデル
net = SSD(phase="train", cfg=ssd_cfg)

# SSDの初期の重みを設定
# ssdのvgg部分に重みをロードする
vgg_weights = torch.load('./weights/vgg16_reducedfc.pth')
net.vgg.load_state_dict(vgg_weights)

# ssdのその他のネットワークの重みはHeの初期値で初期化


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:  # バイアス項がある場合
            nn.init.constant_(m.bias, 0.0)


# Heの初期値を適用
net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)

# GPUが使えるかを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

print('ネットワーク設定完了：学習済みの重みをロードしました')


使用デバイス： cuda:0
ネットワーク設定完了：学習済みの重みをロードしました


# 損失関数と最適化手法を定義する

In [6]:
from utils.ssd_model import MultiBoxLoss

# 損失関数の設定
criterion = MultiBoxLoss(jaccard_thresh=0.5, neg_pos=3, device=device)

# 最適化手法の設定
optimizer = optim.SGD(net.parameters(), lr=1e-3,
                      momentum=0.9, weight_decay=5e-4)


# 学習・検証を実施する

In [7]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # イタレーションカウンタをセット
    iteration = 1
    epoch_train_loss = 0.0  # epochの損失和
    epoch_val_loss = 0.0  # epochの損失和
    logs = []

    # epochのループ
    for epoch in range(num_epochs+1):

        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
                print('（train）')
            else:
                if((epoch+1) % 10 == 0):
                    net.eval()   # モデルを検証モードに
                    print('-------------')
                    print('（val）')
                else:
                    # 検証は10回に1回だけ行う
                    continue

            # データローダーからminibatchずつ取り出すループ
            for images, targets in dataloaders_dict[phase]:

                # GPUが使えるならGPUにデータを送る
                images = images.to(device)
                targets = [ann.to(device)
                           for ann in targets]  # リストの各要素のテンソルをGPUへ

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    # 順伝搬（forward）計算
                    outputs = net(images)

                    # 損失の計算
                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()  # 勾配の計算

                        # 勾配が大きくなりすぎると計算が不安定になるので、clipで最大でも勾配2.0に留める
                        nn.utils.clip_grad_value_(
                            net.parameters(), clip_value=2.0)

                        optimizer.step()  # パラメータ更新

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('イタレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item(), duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    # 検証時
                    else:
                        epoch_val_loss += loss.item()

        # epochのphaseごとのlossと正解率
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss, epoch_val_loss))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        # ログを保存
        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss, 'val_loss': epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

        epoch_train_loss = 0.0  # epochの損失和
        epoch_val_loss = 0.0  # epochの損失和

        # ネットワークを保存する
        if ((epoch+1) % 10 == 0):
            torch.save(net.state_dict(), 'weights/ssd300_' +
                       str(epoch+1) + '.pth')


In [8]:
# 学習・検証を実行する
num_epochs= 50  
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

使用デバイス： cuda:0
-------------
Epoch 1/50
-------------
（train）
イタレーション 10 || Loss: 16.7855 || 10iter: 56.8422 sec.
イタレーション 20 || Loss: 12.0645 || 10iter: 25.4445 sec.
イタレーション 30 || Loss: 12.1115 || 10iter: 25.7550 sec.
イタレーション 40 || Loss: 9.4758 || 10iter: 25.9485 sec.
イタレーション 50 || Loss: 8.8132 || 10iter: 25.5214 sec.
イタレーション 60 || Loss: 8.6267 || 10iter: 25.6975 sec.
イタレーション 70 || Loss: 8.9257 || 10iter: 25.3736 sec.
イタレーション 80 || Loss: 8.8706 || 10iter: 26.0773 sec.
イタレーション 90 || Loss: 8.5930 || 10iter: 25.6935 sec.
イタレーション 100 || Loss: 8.2677 || 10iter: 25.0656 sec.
イタレーション 110 || Loss: 8.4029 || 10iter: 26.9894 sec.
イタレーション 120 || Loss: 8.0918 || 10iter: 26.3040 sec.
イタレーション 130 || Loss: 7.8098 || 10iter: 25.5423 sec.
イタレーション 140 || Loss: 7.8255 || 10iter: 25.1672 sec.
イタレーション 150 || Loss: 7.9216 || 10iter: 24.8809 sec.
イタレーション 160 || Loss: 7.7873 || 10iter: 25.6822 sec.
イタレーション 170 || Loss: 7.4295 || 10iter: 25.4814 sec.
-------------
epoch 1 || Epoch_TRAIN_Loss:1687.4547 ||Epoch_

イタレーション 1380 || Loss: 5.9921 || 10iter: 26.7026 sec.
イタレーション 1390 || Loss: 5.5947 || 10iter: 25.7720 sec.
イタレーション 1400 || Loss: 5.6044 || 10iter: 25.1158 sec.
イタレーション 1410 || Loss: 5.8772 || 10iter: 25.8379 sec.
イタレーション 1420 || Loss: 5.3639 || 10iter: 25.6109 sec.
イタレーション 1430 || Loss: 5.1371 || 10iter: 25.1652 sec.
-------------
epoch 8 || Epoch_TRAIN_Loss:997.8303 ||Epoch_VAL_Loss:0.0000
timer:  482.8033 sec.
-------------
Epoch 9/50
-------------
（train）
イタレーション 1440 || Loss: 5.6758 || 10iter: 19.6121 sec.
イタレーション 1450 || Loss: 5.1136 || 10iter: 24.9955 sec.
イタレーション 1460 || Loss: 5.5549 || 10iter: 25.0028 sec.
イタレーション 1470 || Loss: 5.7108 || 10iter: 25.2885 sec.
イタレーション 1480 || Loss: 5.6515 || 10iter: 25.5272 sec.
イタレーション 1490 || Loss: 5.0765 || 10iter: 25.5207 sec.
イタレーション 1500 || Loss: 5.4318 || 10iter: 25.2657 sec.
イタレーション 1510 || Loss: 4.9409 || 10iter: 26.0861 sec.
イタレーション 1520 || Loss: 5.3402 || 10iter: 24.6531 sec.
イタレーション 1530 || Loss: 4.9482 || 10iter: 25.7842 sec.
イタレーション 

イタレーション 2710 || Loss: 4.1494 || 10iter: 25.0088 sec.
イタレーション 2720 || Loss: 4.7090 || 10iter: 25.4237 sec.
イタレーション 2730 || Loss: 4.7945 || 10iter: 25.7655 sec.
イタレーション 2740 || Loss: 5.0857 || 10iter: 25.6857 sec.
イタレーション 2750 || Loss: 4.9708 || 10iter: 25.3218 sec.
イタレーション 2760 || Loss: 4.6634 || 10iter: 25.7309 sec.
イタレーション 2770 || Loss: 4.5753 || 10iter: 25.5695 sec.
イタレーション 2780 || Loss: 4.8519 || 10iter: 24.9951 sec.
イタレーション 2790 || Loss: 4.9411 || 10iter: 25.6235 sec.
イタレーション 2800 || Loss: 4.5280 || 10iter: 25.6614 sec.
イタレーション 2810 || Loss: 4.8862 || 10iter: 26.3581 sec.
イタレーション 2820 || Loss: 4.7268 || 10iter: 25.9526 sec.
イタレーション 2830 || Loss: 5.3743 || 10iter: 26.3418 sec.
イタレーション 2840 || Loss: 4.9574 || 10iter: 26.2892 sec.
イタレーション 2850 || Loss: 4.1022 || 10iter: 25.7205 sec.
イタレーション 2860 || Loss: 4.7474 || 10iter: 25.4410 sec.
-------------
epoch 16 || Epoch_TRAIN_Loss:841.4152 ||Epoch_VAL_Loss:0.0000
timer:  482.9591 sec.
-------------
Epoch 17/50
-------------
（train）
イタレーショ

イタレーション 4060 || Loss: 4.2875 || 10iter: 25.1917 sec.
イタレーション 4070 || Loss: 4.1426 || 10iter: 25.5065 sec.
イタレーション 4080 || Loss: 4.0500 || 10iter: 25.9946 sec.
イタレーション 4090 || Loss: 4.6800 || 10iter: 25.4530 sec.
イタレーション 4100 || Loss: 4.4089 || 10iter: 25.5338 sec.
イタレーション 4110 || Loss: 4.4038 || 10iter: 25.0739 sec.
-------------
epoch 23 || Epoch_TRAIN_Loss:767.6410 ||Epoch_VAL_Loss:0.0000
timer:  479.1745 sec.
-------------
Epoch 24/50
-------------
（train）
イタレーション 4120 || Loss: 4.0701 || 10iter: 7.1682 sec.
イタレーション 4130 || Loss: 4.4306 || 10iter: 26.0500 sec.
イタレーション 4140 || Loss: 4.1846 || 10iter: 25.8052 sec.
イタレーション 4150 || Loss: 3.9473 || 10iter: 25.3232 sec.
イタレーション 4160 || Loss: 4.0145 || 10iter: 25.0338 sec.
イタレーション 4170 || Loss: 4.3430 || 10iter: 25.9464 sec.
イタレーション 4180 || Loss: 4.1350 || 10iter: 25.5819 sec.
イタレーション 4190 || Loss: 4.3209 || 10iter: 26.2686 sec.
イタレーション 4200 || Loss: 4.1105 || 10iter: 25.9898 sec.
イタレーション 4210 || Loss: 4.3594 || 10iter: 26.3413 sec.
イタレーション

イタレーション 5390 || Loss: 4.0739 || 10iter: 26.2677 sec.
イタレーション 5400 || Loss: 3.7981 || 10iter: 24.8667 sec.
イタレーション 5410 || Loss: 3.6619 || 10iter: 25.4550 sec.
イタレーション 5420 || Loss: 4.0348 || 10iter: 24.5696 sec.
イタレーション 5430 || Loss: 3.9663 || 10iter: 25.2371 sec.
イタレーション 5440 || Loss: 4.6113 || 10iter: 24.9540 sec.
イタレーション 5450 || Loss: 3.6903 || 10iter: 25.4988 sec.
イタレーション 5460 || Loss: 4.3158 || 10iter: 25.9933 sec.
イタレーション 5470 || Loss: 4.6988 || 10iter: 25.2086 sec.
イタレーション 5480 || Loss: 3.7591 || 10iter: 25.6933 sec.
イタレーション 5490 || Loss: 3.8438 || 10iter: 26.0200 sec.
イタレーション 5500 || Loss: 3.2742 || 10iter: 26.2515 sec.
イタレーション 5510 || Loss: 3.2949 || 10iter: 25.1354 sec.
イタレーション 5520 || Loss: 4.1684 || 10iter: 25.4092 sec.
イタレーション 5530 || Loss: 3.7076 || 10iter: 25.5768 sec.
イタレーション 5540 || Loss: 3.7669 || 10iter: 25.3620 sec.
-------------
epoch 31 || Epoch_TRAIN_Loss:704.2547 ||Epoch_VAL_Loss:0.0000
timer:  480.5075 sec.
-------------
Epoch 32/50
-------------
（train）
イタレーショ

イタレーション 6750 || Loss: 3.6086 || 10iter: 25.6359 sec.
イタレーション 6760 || Loss: 3.7182 || 10iter: 25.8616 sec.
イタレーション 6770 || Loss: 3.3856 || 10iter: 25.1818 sec.
イタレーション 6780 || Loss: 3.6041 || 10iter: 25.8577 sec.
イタレーション 6790 || Loss: 3.9333 || 10iter: 25.9851 sec.
イタレーション 6800 || Loss: 3.4793 || 10iter: 25.9723 sec.
-------------
epoch 38 || Epoch_TRAIN_Loss:666.6890 ||Epoch_VAL_Loss:0.0000
timer:  488.0061 sec.
-------------
Epoch 39/50
-------------
（train）
イタレーション 6810 || Loss: 4.1479 || 10iter: 20.2294 sec.
イタレーション 6820 || Loss: 3.5176 || 10iter: 25.9758 sec.
イタレーション 6830 || Loss: 3.5798 || 10iter: 25.1273 sec.
イタレーション 6840 || Loss: 3.6164 || 10iter: 25.7199 sec.
イタレーション 6850 || Loss: 3.8225 || 10iter: 25.8477 sec.
イタレーション 6860 || Loss: 3.6669 || 10iter: 25.5076 sec.
イタレーション 6870 || Loss: 3.5214 || 10iter: 26.1864 sec.
イタレーション 6880 || Loss: 3.8865 || 10iter: 25.4400 sec.
イタレーション 6890 || Loss: 3.7801 || 10iter: 25.8262 sec.
イタレーション 6900 || Loss: 3.5785 || 10iter: 24.9307 sec.
イタレーショ

イタレーション 8080 || Loss: 3.1628 || 10iter: 26.2816 sec.
イタレーション 8090 || Loss: 3.4439 || 10iter: 25.2164 sec.
イタレーション 8100 || Loss: 3.5322 || 10iter: 26.5406 sec.
イタレーション 8110 || Loss: 3.9425 || 10iter: 25.9930 sec.
イタレーション 8120 || Loss: 3.5517 || 10iter: 26.0529 sec.
イタレーション 8130 || Loss: 2.8094 || 10iter: 26.1619 sec.
イタレーション 8140 || Loss: 3.7260 || 10iter: 25.6256 sec.
イタレーション 8150 || Loss: 3.7778 || 10iter: 25.9643 sec.
イタレーション 8160 || Loss: 3.5408 || 10iter: 26.8926 sec.
イタレーション 8170 || Loss: 3.9817 || 10iter: 26.5341 sec.
イタレーション 8180 || Loss: 3.5449 || 10iter: 25.6559 sec.
イタレーション 8190 || Loss: 3.5353 || 10iter: 25.4507 sec.
イタレーション 8200 || Loss: 3.4015 || 10iter: 25.0588 sec.
イタレーション 8210 || Loss: 3.2838 || 10iter: 25.8151 sec.
イタレーション 8220 || Loss: 3.3467 || 10iter: 25.5586 sec.
イタレーション 8230 || Loss: 3.0808 || 10iter: 25.3809 sec.
-------------
epoch 46 || Epoch_TRAIN_Loss:632.5500 ||Epoch_VAL_Loss:0.0000
timer:  488.2781 sec.
-------------
Epoch 47/50
-------------
（train）
イタレーショ

以上